# Introduction

Here, we will demonstrate the use of generalized additive models and trees. For this work, we will be reusing our Lab 2 dataset. I will reiterate that cross-validation must be done to have any confidence in your ML models. At the very least, a proper training/test split should be done.

As we move towards the end of this course, we are going to be working with much more sophisticated tools in the scikit-learn package. Specifically, one of the things we will use extensively is [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV). This class allows us to scan a set of parameter values for a model and return the CV results.

In [1]:
import collections

from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

from pymatgen.core import Element, Composition

rcparams = {'legend.fontsize': 20,
            'figure.figsize': (12, 8),
            'axes.labelsize': 24,
            'axes.titlesize': 28,
            'xtick.labelsize':20,
            'ytick.labelsize': 20}
sns.set(rc=rcparams)
mpl.rcParams.update(rcparams)

%matplotlib inline
%config InlineBackend.figure_format ='retina'

from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier, \
    GradientBoostingClassifier, GradientBoostingRegressor, \
    RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import export_text, export_graphviz

from statsmodels.gam.generalized_additive_model import GLMGam
from statsmodels.gam.api import BSplines
import statsmodels.api as sm

In [2]:
# by default pandas will recognize NaN (sodium nitride) as nan (not a number)
# to turn off this behavior, we use na_filter=False
data_url = "https://raw.githubusercontent.com/materialsvirtuallab/nano281/master/labs/lab2/data.csv"
data = pd.read_csv(data_url, index_col=0, na_filter=False)

# Let's create a column of Composition objects using pymatgen.
data["composition"] = [Composition(f) for f in data["formula"]]

Here, we load the elemental data. Unlike lab2, we are simply going to disregard all elemental features that contain NaN. While imputing the mean value is a commonly used data science technique, it really does not work well for materials science problems. We will also use a smaller set of properties.

In [3]:
el_data_url = "https://raw.githubusercontent.com/materialsvirtuallab/nano281/master/labs/lab2/element_properties.csv"
el_data = pd.read_csv(el_data_url, index_col=0)
el_data = el_data[['AtomicRadius', 'AtomicWeight', 'Column', 'Electronegativity', 'Row']]

As before, we will compute the mean, min, and max for every elemental feature. For mean, we are weighting it by composition. As before, we drop all data points that contain NaN.

In [4]:
props = collections.defaultdict(list)

for comp in data["composition"]:
    for c in el_data.columns:
        vals = [el_data[c][el.symbol] for el, amt in comp.items()]
        comp_vals = [el_data[c][el.symbol] * amt for el, amt in comp.items()]
        props["%sMean" % c].append(sum(comp_vals)/ comp.num_atoms)
        props["%sMin" % c].append(min(vals))
        props["%sMax" % c].append(max(vals))
data = data.assign(**props)
data = data.dropna()
print(data.shape)

(124342, 21)


We are left with around 106k data points, Still more than enough for our purposes. Let's create our features and targets.

In [5]:
features = [c for c in data.columns if c.endswith("Mean") or c.endswith("Min") or c.endswith("Max")]
x = data[features]
y_class = [0 if bg < 1e-4 else 1 for bg in data["band_gap"]]
y_reg = data["band_gap"]

Before we proceed further, let us write up some reusable methods to standardize the analysis of different ML models. Copy and pasting code is fine for earlier demos to reiterate the API of scikit-learn, but it is very bad programming practice. By this stage of the course, we want to do things better.

In [6]:
def plot_grid_search_results(gs, ylim=None):
    """
    Plots the results of GridSearchCV.
    
    Args:
        gs: A GridSearchCV object.
        ylim: Optional setting for y limits.
    """
    results = pd.DataFrame(gs.cv_results_)
    for c in results.columns:
        # Note that here we are working with just variations in one parameter.
        # So we can automatically find the name of that parameter.
        if c.startswith("param_"):
            x = c
            break
    fig, ax = plt.subplots(figsize=(16, 8))    
    ax = sns.lineplot(x=x, y="mean_train_score", data=results)
    ax = sns.scatterplot(x=x, y="mean_train_score", data=results, marker='x')
    ax = sns.lineplot(x=x, y="mean_test_score", data=results)
    ax = sns.scatterplot(x=x, y="mean_test_score", data=results, marker='o')
    plt.xlabel(x)
    if ylim:
        plt.ylim(ylim)
    ax.legend(["Train", "Test"], loc=2);
    
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Generalized Additive Models

Since sklearn does not have generalized additive models, we will use statsmodels for fitting such models. 

In [7]:
# GAM model is difficult to train, here we use a very small train data size 
x_train, x_test, y_train, y_test = train_test_split(x, y_class, test_size=0.9, random_state=42)

# smoother over the 15 variables, with each variable using 6 basis function (the df parameter)
# and degree of 4 (the degree parameter) splines. 
bs = BSplines(x_train, df=[6] * 15, degree=[4] * 15 )

In [8]:
# Create the training data frame, which contains the features and the target, i.e., is_metal
combined_xy = x_train.copy()
combined_xy = combined_xy.assign(**{'is_metal': y_train})

We know that the target `is_metal` is either 0 or 1, which follows a Berloulli distribution. The Berloulli distribution can be seen as a special case of Binomial distribution with total number of trial of 1. 

Therefore, we model the target $Y$ (i.e., `is_metal`) as a Binomial distribution. 

$g(E(Y)) = \beta_0 + \sum_i^Kf_i(x_i)$

$\beta_0$ is the intercept, $i$ means the i-th variable, and $f_i(x_i)$ is modeled as a smooth function. In our case, the smooth function is B-spline defined earlier. 

In order to reproduce the correct distribution of $Y$, we set the link function $g$ to a logit function. 

$g(x) = \log\frac{x}{1-x}$

In [9]:
# Gaussian family with Logit link function

binomial = sm.families.Binomial(sm.families.links.logit)
gam_bs = GLMGam.from_formula('is_metal ~ 1', data=combined_xy, 
                             smoother=bs, family=binomial)

TypeError: Calling Family(..) with a link class is not allowed. Use an instance of a link class instead.

The formula means that other than the generalized additive models using B-Splines, there is only an extra intercept term (1 in "is_metal ~ 1").

We can of course include other linear relationship here. For example, if we believe there is an extra linear dependence on AtomicRadiusMean, we should write the formula as 

"is_metal ~ AtomicRadiusMean + 1"

Note that the generalized additive models are added to the relationship.

In [ ]:
%time res_bs = gam_bs.fit()

In [ ]:
y_pred_test = []
y_test_valid = []

# some test x is outside the training x bounds, which will cause errors, 
# we use try... except to ignore this error
for x_test_temp, y_test_temp in zip(x_test.values, y_test):
    try:
        transformed = bs.transform(x_test_temp.reshape((1, -1)))
        transformed = np.concatenate([np.array([[1]]), transformed], axis=1)  # add intercept
        y_pred_test.append(gam_bs.predict(res_bs.params, transformed).item())
        y_test_valid.append(y_test_temp)
    except:
        pass

print((f"The prediction accuracy on {len(y_test_valid)} test data is "
       f"{accuracy_score(np.array(y_pred_test)>=0.5, y_test_valid):.3f}"))

# Decision tree classifier

Here, we will construct a decision tree classifier. Let us explore how the classficication accuracy changes with the tree depth. We will use an extremely powerful tool in the scikit-learn toolkit called GridSearchCV, which automatically varies a parameter across a set of values and returns the CV results!

In [ ]:
gs = GridSearchCV(
    DecisionTreeClassifier(random_state=0),
    param_grid={"max_depth": list(range(1, 20))},
    return_train_score=True,
    scoring="accuracy",
    cv=kfold
)
gs.fit(x, y_class)

In [ ]:
gs.cv_results_

In [ ]:
plot_grid_search_results(gs)
plt.ylabel("accuracy");

We see the test and training accuracy diverges after a tree depth of 8 or so, and the test accuracy plateaus after a tree depth of 15 or so. A relatively good accuracy of around 80% can be achieved. Let's visualize the tree structure.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_class, test_size=0.2)

decision_tree = DecisionTreeClassifier(criterion="entropy", random_state=0, max_depth=8)
decision_tree.fit(x_train, y_train)

# We can export this to a graphviz dot file, which can be used to generate a nice plot.
export_graphviz(decision_tree, out_file="metal_insulator_tree.dot", feature_names=list(x.columns))

Let's see what happens when we change the pruning parameter.

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=8)
gs = GridSearchCV(
    decision_tree,
    param_grid={"ccp_alpha": [1e-4, 1e-3, 1e-2]},
    return_train_score=True,
    scoring="accuracy",
    cv=kfold
)
gs.fit(x, y_class)
plot_grid_search_results(gs)
plt.ylabel("accuracy")
plt.xscale('log');

As we increase $\alpha$, we decrease the accuracy, but the tree depth decreases.

Let us now visualize a smaller tree to see how decisions are made.

In [ ]:
decision_tree = DecisionTreeClassifier(criterion="entropy", random_state=0, max_depth=3)
decision_tree.fit(x_train, y_train)
r = export_text(decision_tree, feature_names=list(x.columns))
print(r)

## Feature importance

A very useful of decision trees is that we can visualize the feature importance quite easily.

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=8)
decision_tree.fit(x_train, y_train)
plt.subplots(figsize=(16, 16))
sns.barplot(decision_tree.feature_importances_, list(x.columns), orient='h')
plt.xlabel("Feature importance");

## Receiver Operating Characteristic Curve

In [ ]:
plt.subplots(figsize=(16, 16))
for d in [4, 8, 16, 32]:
    decision_tree = DecisionTreeClassifier(random_state=0, max_depth=d)
    decision_tree = decision_tree.fit(x_train, y_train)
    train_accuracy = decision_tree.score(x_train, y_train)
    test_accuracy = decision_tree.score(x_test, y_test)

    y_pred = decision_tree.predict_proba(x_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    a = roc_auc_score(y_test, y_pred)
    plt.plot(fpr, tpr, 'o-', label='d = %d, AUC = %.3f' % (d, a))

plt.plot([0, 0, 1], [0, 1, 1], 'k-', label="Ideal")
plt.plot([0, 1], [0, 1], 'k-.', label="Random guess")
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best');

# Decision Tree Regressor

Instead of just classifying metals and insulators, let us now use the decision tree for regression instead. Here, we will use MSE instead of classification accuracy as the criterion.

In [ ]:
decision_tree = DecisionTreeRegressor(random_state=0)
gs = GridSearchCV(
    decision_tree,
    param_grid={"max_depth": range(1, 20)},
    return_train_score=True,
    scoring="neg_mean_squared_error",
    cv=kfold
)
gs.fit(x, y_reg)
plot_grid_search_results(gs)
plt.ylabel("-MSE");

We observe that an optimal tree depth of around 15 or so in terms of minimizing MSE. We can achieve a MSE of slightly more than 1 eV. Not great, but reasonable for such a simple model. Let us now explore how alpha affects the tree depth and the MSE.

In [ ]:
decision_tree = DecisionTreeRegressor(criterion="mse", random_state=0, max_depth=15)

gs = GridSearchCV(
    decision_tree,
    param_grid={"ccp_alpha": np.logspace(-4, -1, 10)},
    return_train_score=True,
    scoring="neg_mean_squared_error",
    cv=kfold
)
gs.fit(x, y_reg)
plot_grid_search_results(gs)
plt.ylabel("- MSE")
plt.xscale('log');

As we increase $\alpha$, we get a simpler (shallower) tree, but the MSE increases. However, the training and test error converges and we get less overfitting.

Let's look at a relatively small tree.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_reg, test_size=0.2)

decision_tree = DecisionTreeRegressor(criterion="mse", random_state=0, max_depth=15, ccp_alpha=0.01)
decision_tree.fit(x_train, y_train)
y_pred = decision_tree.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r = export_text(decision_tree, feature_names=list(x.columns))
print("MSE = %.3f" % (mse))
print(r)

# Ensemble Learning

Here, we are going to look at some ensemble learning approaches to improve the predictions of decision trees for both classification and regression. To demonstrate the impact more clearly, we will use relatively shallow trees of `max_depth=8` for classification and `max_depth=15` for regression throughout.

In [ ]:
max_depth_class = 3
max_depth_reg = 15

## Adaboost

In [ ]:
model = AdaBoostClassifier(DecisionTreeClassifier(random_state=0, max_depth=max_depth_class))

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": range(10, 100, 20)},
    return_train_score=True,
    scoring="accuracy",
    cv=kfold
)
gs.fit(x, y_class)
plot_grid_search_results(gs)
plt.ylabel("accuracy");

We can see that even with a small tree, boosting allows us to achieve higher classification accuracies. There is some evidence of overfitting, as can be seen from the divergence of the training and test accuracies. We can of course play around with the tree depth as well as the learning rate to improve the results.

For regression, we will use a decision tree with a max depth of 15 and a lower learning rate.

In [ ]:
model = AdaBoostRegressor(DecisionTreeRegressor(random_state=0, max_depth=max_depth_reg), 
                          learning_rate=0.1)

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": [2, 4, 8, 16, 32, 64]},
    return_train_score=True,
    scoring="neg_mean_squared_error",
    cv=kfold
)
gs.fit(x, y_reg)
plot_grid_search_results(gs)
plt.ylabel("- MSE");

We can see that we can get to a substantially lower test MSE of 0.85 eV or so with boosting, albeit with some evidence of over-fitting.

## Gradient boosting

In [ ]:
model = GradientBoostingClassifier(random_state=0, max_depth=max_depth_class)

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": range(10, 800, 100)},
    return_train_score=True,
    scoring="accuracy",
    cv=kfold,
    n_jobs=4
)
gs.fit(x, y_class)
plot_grid_search_results(gs)
plt.ylabel("accuracy");

In [ ]:
model = GradientBoostingRegressor(random_state=0, max_depth=max_depth_reg)

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": range(10, 100, 20)},
    return_train_score=True,
    scoring="neg_mean_squared_error",
    cv=kfold,
    n_jobs=4
)
gs.fit(x, y_reg)
plot_grid_search_results(gs)
plt.ylabel("-MSE");

## Random Forests

We will now lookn at random forest models for performing the same classificaiton and regression tasks. For classification, we will first use the same maximum tree depth as before of 3.

In [ ]:
model = RandomForestClassifier(random_state=0, max_depth=max_depth_class)

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": range(20, 120, 20)},
    return_train_score=True,
    scoring="accuracy",
    cv=kfold,
    n_jobs=4
)
gs.fit(x, y_class)
plot_grid_search_results(gs)
plt.ylabel("accuracy");

We see that with a maximum depth of 3, there is very little overfitting, but the performance is somewhat lower that what we have achieved with gradient boosting.

Let us now remove all variables and let the default implementation try to find the optimal random forest model.

In [ ]:
model = RandomForestClassifier(random_state=0)

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": range(1, 40, 4)},
    return_train_score=True,
    scoring="accuracy",
    cv=kfold,
    n_jobs=4
)
gs.fit(x, y_class)
plot_grid_search_results(gs)
plt.ylabel("accuracy");

We see that the performance has substantially improved for both training and test data, but there is evidence of significant overfitting!

We can gain some insights into how the random forest is getting this performance by looking at the feature importances. 

In [ ]:
model = RandomForestClassifier(random_state=0, n_estimators=20)
model.fit(x, y_class)
plt.subplots(figsize=(16, 16))
sns.barplot(model.feature_importances_, list(x.columns), orient='h')
plt.xlabel("Feature importance");

We will now look at the random forest for the regression task.

In [ ]:
model = RandomForestRegressor(random_state=0, max_depth=max_depth_reg)

gs = GridSearchCV(
    model,
    param_grid={"n_estimators": range(1, 20, 2)},
    return_train_score=True,
    scoring="neg_mean_squared_error",
    cv=kfold,
    n_jobs=8
)
gs.fit(x, y_reg)
plot_grid_search_results(gs)
plt.ylabel("-MSE");

With the same max depth, we see that the performance has substantially improved. A test MSE of < 1eV is now possible but again, we see quite a bit of overfitting. Let's try to gain some insight into the base decision tree being used with 10 estimators.

While you can try to further tweak he model by setting parameters like `min_samples leaf`, a simpler approach is to just let the algorithm automatically prune the tree by setting the $\alpha$ parameter.

In [ ]:
model = RandomForestRegressor(random_state=0, max_depth=15, n_estimators=10)

gs = GridSearchCV(
    model,
    param_grid={"ccp_alpha": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]},
    return_train_score=True,
    scoring="neg_mean_squared_error",
    cv=kfold,
    n_jobs=8
)
gs.fit(x, y_reg)
plot_grid_search_results(gs)
plt.ylabel("-MSE");
plt.xscale('log')

Quite clearly, there is a tradeoff between tree complexity, MSE and overfitting. At high $\alpha > 1e-3$, there is minimal overfitting but the best we can do is an MSE of around 1.2 eV or more. The less the overfitting, the more confidence we have about the model being generalizable to future unseen data.